# CH07-08: LLM 實際應用案例

**課程時長**: 90 分鐘  
**難度**: ⭐⭐⭐⭐⭐  
**前置知識**: CH07-01 至 CH07-07  

---

## 📚 本節學習目標

1. ✅ 使用 Hugging Face Transformers 載入與使用 LLM
2. ✅ 實作檢索增強生成 (RAG) 系統
3. ✅ 掌握 LangChain 框架核心功能
4. ✅ 設計 LLM Agent 系統
5. ✅ 理解生產環境部署考量

---

## 📖 目錄

1. [使用 Hugging Face 載入 LLM](#1-huggingface)
2. [文本生成實戰](#2-text-generation)
3. [檢索增強生成 (RAG)](#3-rag)
4. [LangChain 框架入門](#4-langchain)
5. [Agent 系統設計](#5-agents)
6. [生產環境部署](#6-deployment)
7. [總結](#7-summary)

---

## 1. 使用 Hugging Face 載入 LLM {#1-huggingface}

### 1.1 Hugging Face Hub 簡介

**Hugging Face** 是最大的開源 AI 社群:
- 200,000+ 預訓練模型
- 50,000+ 數據集
- 統一的 API 介面

**熱門開源 LLM**:
- **LLaMA 2** (Meta): 7B, 13B, 70B
- **Mistral** (Mistral AI): 7B, 8x7B (MoE)
- **Falcon** (TII): 7B, 40B, 180B
- **MPT** (MosaicML): 7B, 30B
- **Bloom** (BigScience): 176B

### 1.2 安裝必要套件

In [ ]:
# 安裝必要套件 (取消註解以執行)
# !pip install transformers>=4.35.0
# !pip install torch>=2.0.0
# !pip install accelerate>=0.20.0
# !pip install bitsandbytes>=0.40.0  # 用於量化
# !pip install sentence-transformers>=2.2.0  # 用於 embeddings
# !pip install faiss-cpu>=1.7.4  # 用於向量檢索
# !pip install langchain>=0.0.300  # LangChain 框架

print("✅ 套件安裝指令已準備好")
print("\n注意: 實際執行前請確認環境，建議使用虛擬環境")

In [ ]:
# 載入基礎套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

print("✅ 基礎套件載入完成")

# 嘗試載入 Transformers (可能未安裝)
try:
    from transformers import (
        AutoModelForCausalLM,
        AutoTokenizer,
        pipeline,
        BitsAndBytesConfig
    )
    import torch
    
    print(f"✅ Transformers {transformers.__version__} 載入成功")
    print(f"✅ PyTorch {torch.__version__} 載入成功")
    print(f"✅ CUDA 可用: {torch.cuda.is_available()}")
    
    HF_AVAILABLE = True
    
except ImportError as e:
    print("⚠️ Transformers 或 PyTorch 未安裝")
    print("   請執行: pip install transformers torch")
    HF_AVAILABLE = False

### 1.3 載入小型 LLM (GPT-2)

In [ ]:
if HF_AVAILABLE:
    print("載入 GPT-2 Small (124M 參數)...")
    
    # 載入 tokenizer 和模型
    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # 移到 GPU (如果可用)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    
    print(f"✅ GPT-2 載入完成 (Device: {device})")
    print(f"✅ Tokenizer 詞彙量: {len(tokenizer)}")
    print(f"✅ 模型參數量: {model.num_parameters():,}")
    
    # 測試生成
    prompt = "Artificial intelligence is"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=30,
        num_return_sequences=1,
        temperature=0.7,
        do_sample=True
    )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print("\n生成範例:")
    print(f"Prompt: {prompt}")
    print(f"Output: {generated_text}")
    
else:
    print("⚠️ 請先安裝 transformers 和 torch")

## 2. 文本生成實戰 {#2-text-generation}

### 2.1 生成參數詳解

**關鍵參數**:

| 參數 | 說明 | 推薦值 |
|------|------|--------|
| **max_new_tokens** | 生成的最大 token 數 | 50-500 |
| **temperature** | 隨機性控制 (越高越隨機) | 0.7-1.0 (創意), 0.1-0.3 (事實) |
| **top_k** | 只從 top-k 個 token 中採樣 | 50 |
| **top_p** | Nucleus sampling (累積機率) | 0.9-0.95 |
| **num_beams** | Beam search 寬度 | 1 (greedy), 4-5 (品質優先) |
| **do_sample** | 是否隨機採樣 | True (多樣), False (確定性) |
| **repetition_penalty** | 重複懲罰 | 1.2-1.5 |

### 2.2 不同生成策略對比

In [ ]:
if HF_AVAILABLE:
    prompt = "The future of artificial intelligence will"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    strategies = [
        {
            'name': 'Greedy (確定性)',
            'params': {
                'max_new_tokens': 40,
                'do_sample': False
            }
        },
        {
            'name': 'Sampling (隨機)',
            'params': {
                'max_new_tokens': 40,
                'do_sample': True,
                'temperature': 1.0
            }
        },
        {
            'name': 'Top-k Sampling',
            'params': {
                'max_new_tokens': 40,
                'do_sample': True,
                'top_k': 50,
                'temperature': 0.8
            }
        },
        {
            'name': 'Nucleus (Top-p) Sampling',
            'params': {
                'max_new_tokens': 40,
                'do_sample': True,
                'top_p': 0.9,
                'temperature': 0.8
            }
        },
        {
            'name': 'Beam Search',
            'params': {
                'max_new_tokens': 40,
                'num_beams': 5,
                'early_stopping': True
            }
        }
    ]
    
    print("=" * 80)
    print(f"Prompt: '{prompt}'")
    print("=" * 80)
    print()
    
    for strategy in strategies:
        outputs = model.generate(**inputs, **strategy['params'])
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        print(f"【{strategy['name']}】")
        print(f"{text}")
        print()
    
    print("=" * 80)
    print("\n觀察:")
    print("• Greedy: 每次都生成相同結果")
    print("• Sampling: 每次結果不同,更有創意")
    print("• Top-k/Top-p: 平衡品質與多樣性")
    print("• Beam Search: 品質最高但缺乏多樣性")

else:
    print("⚠️ 需要安裝 transformers")

### 2.3 使用 Pipeline 簡化操作

In [ ]:
if HF_AVAILABLE:
    # 建立文本生成 pipeline
    generator = pipeline(
        "text-generation",
        model=model_name,
        device=0 if torch.cuda.is_available() else -1  # 0=GPU, -1=CPU
    )
    
    # 生成文本
    prompt = "Once upon a time in a magical forest,"
    
    results = generator(
        prompt,
        max_new_tokens=60,
        num_return_sequences=3,  # 生成 3 個版本
        temperature=0.8,
        top_p=0.95,
        do_sample=True
    )
    
    print(f"Prompt: '{prompt}'\n")
    print("生成的 3 個版本:\n")
    
    for i, result in enumerate(results, 1):
        print(f"版本 {i}:")
        print(result['generated_text'])
        print()

else:
    print("⚠️ 需要安裝 transformers")

## 3. 檢索增強生成 (RAG) {#3-rag}

### 3.1 RAG 原理

**問題**: LLM 的知識是靜態的,無法獲取最新資訊

**解決方案**: Retrieval-Augmented Generation (RAG)

```
用戶問題
    ↓
① 檢索相關文檔 (Retrieval)
    ↓
    向量數據庫搜尋
    ↓
② 整合上下文 (Augmentation)
    ↓
    問題 + 檢索到的文檔 → Prompt
    ↓
③ LLM 生成答案 (Generation)
    ↓
   最終答案
```

**優勢**:
- ✅ 獲取最新資訊
- ✅ 減少幻覺
- ✅ 可追溯來源
- ✅ 無需重新訓練

### 3.2 實作簡單 RAG 系統

In [ ]:
# 模擬知識庫
knowledge_base = [
    {
        "id": 1,
        "text": "Transformer 是一種基於自注意力機制的神經網路架構,由 Vaswani 等人於 2017 年提出。它徹底改變了 NLP 領域。"
    },
    {
        "id": 2,
        "text": "BERT 是 Google 在 2018 年發布的預訓練語言模型,使用 Transformer Encoder 架構,在多個 NLP 任務上取得突破性成果。"
    },
    {
        "id": 3,
        "text": "GPT-3 是 OpenAI 於 2020 年發布的大型語言模型,擁有 175B 參數,展現了強大的 few-shot learning 能力。"
    },
    {
        "id": 4,
        "text": "LLaMA 是 Meta 在 2023 年開源的高效 LLM 系列,參數從 7B 到 65B,在相同參數量下表現優於其他開源模型。"
    },
    {
        "id": 5,
        "text": "RAG (檢索增強生成) 結合了檢索系統和生成模型,通過檢索相關文檔來增強 LLM 的回答,減少幻覺問題。"
    }
]

print("✅ 知識庫建立完成")
print(f"   共 {len(knowledge_base)} 筆文檔")

# 簡單的關鍵字檢索 (實際應用中應使用向量檢索)
def simple_retrieve(query, knowledge_base, top_k=2):
    """簡單的關鍵字匹配檢索"""
    scores = []
    query_lower = query.lower()
    
    for doc in knowledge_base:
        # 計算匹配分數 (簡化版)
        text_lower = doc['text'].lower()
        score = sum(1 for word in query_lower.split() if word in text_lower)
        scores.append((doc, score))
    
    # 排序並返回 top-k
    scores.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, score in scores[:top_k]]

# 測試檢索
test_query = "什麼是 Transformer?"
retrieved_docs = simple_retrieve(test_query, knowledge_base)

print(f"\n查詢: '{test_query}'")
print("\n檢索到的文檔:")
for doc in retrieved_docs:
    print(f"  [{doc['id']}] {doc['text']}")

In [ ]:
# RAG 問答函數
def rag_qa(query, knowledge_base, model, tokenizer, top_k=2):
    """
    RAG 問答系統
    
    Args:
        query: 用戶問題
        knowledge_base: 知識庫
        model: LLM 模型
        tokenizer: Tokenizer
        top_k: 檢索文檔數量
    
    Returns:
        答案文本
    """
    # 1. 檢索相關文檔
    docs = simple_retrieve(query, knowledge_base, top_k)
    
    # 2. 構建 prompt
    context = "\n".join([f"- {doc['text']}" for doc in docs])
    
    prompt = f"""根據以下資訊回答問題:

{context}

問題: {query}
答案:"""
    
    print("=" * 80)
    print("構建的 Prompt:")
    print("=" * 80)
    print(prompt)
    print("=" * 80)
    
    # 3. 生成答案
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.3,  # 降低隨機性,更事實性
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 提取答案部分
    answer = answer.split("答案:")[-1].strip()
    
    return answer, docs

# 測試 RAG 系統
if HF_AVAILABLE:
    queries = [
        "什麼是 Transformer?",
        "GPT-3 有多少參數?",
        "BERT 是什麼時候發布的?"
    ]
    
    for query in queries:
        print("\n" + "=" * 80)
        print(f"問題: {query}")
        print("=" * 80)
        
        answer, sources = rag_qa(query, knowledge_base, model, tokenizer)
        
        print(f"\n答案: {answer}")
        print(f"\n來源:")
        for doc in sources:
            print(f"  [{doc['id']}] {doc['text'][:50]}...")

else:
    print("⚠️ 需要載入 LLM 模型")

### 3.3 使用向量檢索 (進階)

**實際 RAG 系統應使用語義相似度檢索**:

```python
# 使用 Sentence Transformers + FAISS
from sentence_transformers import SentenceTransformer
import faiss

# 1. 載入 embedding 模型
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. 將文檔轉換為向量
doc_embeddings = embed_model.encode([doc['text'] for doc in knowledge_base])

# 3. 建立 FAISS 索引
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# 4. 檢索
query_embedding = embed_model.encode([query])
distances, indices = index.search(query_embedding, k=2)
```

**向量檢索 vs 關鍵字檢索**:

| 特性 | 關鍵字檢索 | 向量檢索 |
|------|-----------|----------|
| 精確度 | 低 | 高 |
| 語義理解 | ❌ | ✅ |
| 同義詞 | ❌ | ✅ |
| 速度 | 快 | 需索引加速 |
| 實作複雜度 | 低 | 中 |

## 4. LangChain 框架入門 {#4-langchain}

### 4.1 LangChain 簡介

**LangChain** 是構建 LLM 應用的框架:
- 統一的 LLM 介面
- 鏈式組合 (Chain)
- 記憶管理 (Memory)
- Agent 系統

**核心概念**:

```
LangChain
├── Models (LLM, Chat Model, Embeddings)
├── Prompts (PromptTemplate, ChatPromptTemplate)
├── Chains (LLMChain, SequentialChain, ...)
├── Memory (ConversationBufferMemory, ...)
├── Agents (ReAct, Plan-and-Execute, ...)
└── Tools (Search, Calculator, API calls, ...)
```

### 4.2 LangChain 基礎範例

In [ ]:
# LangChain 範例 (示範程式碼,需安裝 langchain)

langchain_code = '''
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

# 1. 包裝 HuggingFace 模型為 LangChain LLM
llm = HuggingFacePipeline(pipeline=generator)

# 2. 建立 Prompt Template
template = """你是一個友善的 AI 助手。

對話歷史:
{history}

用戶: {input}
助手:"""

prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=template
)

# 3. 建立 Memory
memory = ConversationBufferMemory(
    memory_key="history",
    input_key="input"
)

# 4. 建立 Chain
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True
)

# 5. 多輪對話
print(conversation.predict(input="你好,你是誰?"))
print(conversation.predict(input="你能做什麼?"))
print(conversation.predict(input="剛才我問了什麼?"))  # 測試記憶
'''

print("LangChain 對話系統範例程式碼:")
print("=" * 80)
print(langchain_code)
print("=" * 80)

print("\n說明:")
print("• LLMChain: 組合 LLM + Prompt + Memory")
print("• ConversationBufferMemory: 記住完整對話歷史")
print("• PromptTemplate: 動態填充變數到 prompt")
print("• 自動處理多輪對話的上下文傳遞")

## 5. Agent 系統設計 {#5-agents}

### 5.1 什麼是 Agent?

**定義**:
> LLM Agent 是能夠自主決策、使用工具、執行多步驟任務的智能體。

**核心組件**:

```
Agent
├── LLM (決策大腦)
├── Tools (可用工具)
│   ├── Search (搜尋)
│   ├── Calculator (計算)
│   ├── Database (數據庫)
│   └── API Calls (API 呼叫)
├── Memory (記憶)
└── Executor (執行器)
```

### 5.2 ReAct Agent 架構

**ReAct = Reasoning + Acting**

**流程**:
```
1. Thought: "我需要搜尋最新的股價"
2. Action: Search("AAPL stock price")
3. Observation: "Apple 股價 $180.25"
4. Thought: "我現在知道股價了,可以回答"
5. Final Answer: "Apple 當前股價為 $180.25"
```

**範例 Prompt**:
```
你有以下工具:
- Search: 搜尋網路資訊
- Calculator: 執行數學計算

使用以下格式:

Question: 用戶問題
Thought: 你的思考過程
Action: 工具名稱[工具輸入]
Observation: 工具輸出
... (重複 Thought/Action/Observation)
Thought: 我現在知道最終答案了
Final Answer: 最終答案

Question: Apple 股價是多少?去年同期相比漲了多少?
```

### 5.3 簡單 Agent 實作

In [ ]:
# 簡化版 Agent 實作 (示範概念)

class SimpleAgent:
    """簡化版 ReAct Agent"""
    
    def __init__(self, llm, tools):
        self.llm = llm
        self.tools = {tool.name: tool for tool in tools}
    
    def run(self, question, max_steps=5):
        """執行 Agent"""
        history = []
        
        for step in range(max_steps):
            # 構建 prompt
            prompt = self._build_prompt(question, history)
            
            # LLM 生成 thought + action
            response = self.llm.generate(prompt)
            
            # 解析 response
            if "Final Answer:" in response:
                answer = response.split("Final Answer:")[1].strip()
                return answer
            
            # 執行工具
            action = self._parse_action(response)
            if action:
                tool_name, tool_input = action
                observation = self.tools[tool_name].run(tool_input)
                history.append({
                    "thought": response,
                    "action": action,
                    "observation": observation
                })
        
        return "無法在 max_steps 內找到答案"
    
    def _build_prompt(self, question, history):
        """構建 ReAct prompt"""
        prompt = f"Question: {question}\n"
        
        for h in history:
            prompt += f"Thought: {h['thought']}\n"
            prompt += f"Action: {h['action']}\n"
            prompt += f"Observation: {h['observation']}\n"
        
        prompt += "Thought:"
        return prompt
    
    def _parse_action(self, text):
        """解析 Action"""
        # 簡化版解析
        if "Action:" in text:
            action_str = text.split("Action:")[1].split("\n")[0].strip()
            # 假設格式: ToolName[input]
            tool_name = action_str.split("[")[0].strip()
            tool_input = action_str.split("[")[1].rstrip("]")
            return (tool_name, tool_input)
        return None

# 定義工具
class Tool:
    def __init__(self, name, description, func):
        self.name = name
        self.description = description
        self.func = func
    
    def run(self, input_text):
        return self.func(input_text)

# 範例工具
def calculator(expression):
    try:
        result = eval(expression)
        return str(result)
    except:
        return "計算錯誤"

def search(query):
    # 模擬搜尋
    mock_results = {
        "transformer": "Transformer 是一種神經網路架構,由 Vaswani 等人在 2017 年提出。",
        "gpt": "GPT 是 OpenAI 開發的生成式預訓練 Transformer 模型。"
    }
    for key in mock_results:
        if key in query.lower():
            return mock_results[key]
    return "未找到相關資訊"

tools = [
    Tool("Calculator", "執行數學計算", calculator),
    Tool("Search", "搜尋網路資訊", search)
]

print("✅ SimpleAgent 類別定義完成")
print("\n可用工具:")
for tool in tools:
    print(f"  • {tool.name}: {tool.description}")

## 6. 生產環境部署 {#6-deployment}

### 6.1 部署考量因素

**1. 模型選擇**

| 考量 | 小模型 (< 10B) | 大模型 (> 100B) |
|------|---------------|----------------|
| 延遲 | 低 (< 100ms) | 高 (> 1s) |
| 成本 | 低 | 高 |
| 品質 | 中 | 高 |
| 硬體需求 | CPU/小 GPU | 多 GPU/專用硬體 |

**2. 推論優化技術**

**量化 (Quantization)**:
```python
# 8-bit 量化 (減少 75% 記憶體)
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config
)
```

**批次處理 (Batching)**:
- 將多個請求合併處理
- 提高 GPU 利用率
- 權衡延遲與吞吐量

**KV Cache**:
- 快取已計算的 Key/Value
- 加速自回歸生成
- 減少重複計算

**3. 部署架構**

**選項 A: 本地部署**
```
優點:
• 數據隱私
• 完全控制
• 無 API 費用

缺點:
• 硬體成本高
• 維護負擔
• 需要 ML 工程團隊
```

**選項 B: API 服務 (OpenAI, Anthropic)**
```
優點:
• 即開即用
• 自動擴展
• 最新模型

缺點:
• 按使用量計費
• 數據外流
• 依賴第三方
```

**選項 C: 混合方案**
```
簡單任務 → 本地小模型
複雜任務 → API 大模型
```

### 6.2 監控與維護

**關鍵指標**:

1. **效能指標**
   - 延遲 (P50, P95, P99)
   - 吞吐量 (requests/sec)
   - Token 速度 (tokens/sec)

2. **品質指標**
   - 幻覺率
   - 用戶滿意度
   - 任務成功率

3. **成本指標**
   - 每請求成本
   - GPU 利用率
   - 總擁有成本 (TCO)

**最佳實踐**:

```python
# 1. 設置超時
response = model.generate(..., max_time=30.0)

# 2. 錯誤處理
try:
    result = llm_service.query(prompt)
except TimeoutError:
    result = fallback_response
except Exception as e:
    log_error(e)
    result = error_response

# 3. Rate Limiting
from ratelimit import limits, sleep_and_retry

@sleep_and_retry
@limits(calls=10, period=60)  # 10 requests/min
def call_llm(prompt):
    return model.generate(prompt)

# 4. 快取常見查詢
from functools import lru_cache

@lru_cache(maxsize=1000)
def cached_generate(prompt):
    return model.generate(prompt)
```

In [ ]:
# 部署檢查清單

checklist = {
    '類別': [
        '模型選擇',
        '效能優化',
        '安全性',
        '監控',
        '成本控制'
    ],
    '檢查項目': [
        '✓ 選擇適合任務複雜度的模型規模\n✓ 評估延遲與品質的權衡\n✓ 考慮本地 vs API 部署',
        '✓ 實施量化 (8-bit/4-bit)\n✓ 使用批次處理\n✓ 啟用 KV Cache\n✓ 考慮模型蒸餾',
        '✓ 輸入驗證與過濾\n✓ 輸出內容審核\n✓ Rate Limiting\n✓ 數據加密',
        '✓ 設置效能指標追蹤\n✓ 記錄錯誤與異常\n✓ 監控成本消耗\n✓ A/B 測試新版本',
        '✓ 快取常見查詢\n✓ 使用小模型處理簡單任務\n✓ 優化 prompt 長度\n✓ 設置預算告警'
    ]
}

df_checklist = pd.DataFrame(checklist)

print("=" * 80)
print("LLM 生產環境部署檢查清單")
print("=" * 80)
print()

for _, row in df_checklist.iterrows():
    print(f"【{row['類別']}】")
    print(row['檢查項目'])
    print()

print("=" * 80)

## 7. 總結 {#7-summary}

### 7.1 本節核心要點

✅ **Hugging Face 使用**:
- `AutoModelForCausalLM` + `AutoTokenizer` 載入模型
- `pipeline()` 簡化常見任務
- 掌握生成參數 (temperature, top_k, top_p)

✅ **RAG 系統**:
- 檢索 (Retrieval) → 增強 (Augmentation) → 生成 (Generation)
- 減少幻覺,提供來源追溯
- 向量檢索優於關鍵字檢索

✅ **LangChain 框架**:
- 統一 LLM 介面
- Chain 組合複雜流程
- Memory 管理多輪對話

✅ **Agent 系統**:
- ReAct: Reasoning + Acting
- 工具使用能力
- 多步驟任務分解

✅ **生產部署**:
- 模型選擇: 權衡延遲/成本/品質
- 優化技術: 量化, 批次處理, KV Cache
- 監控: 效能/品質/成本指標

### 7.2 實戰建議

**從小做起**:
1. 先用 GPT-2 / 小型開源模型熟悉流程
2. 在 Colab/Kaggle 免費 GPU 上實驗
3. 掌握基礎後再嘗試大模型

**逐步擴展**:
1. 基礎生成 → RAG → LangChain → Agent
2. 本地開發 → API 測試 → 生產部署
3. 簡單任務 → 複雜應用 → 產品化

**持續學習**:
- 關注 Hugging Face Model Hub 新模型
- 閱讀 LangChain 官方文檔
- 參與開源社群 (Discord, GitHub)

### 7.3 CH07 課程完整總結

**我們學習了什麼**:

1. **CH07-01**: Transformer 架構基礎
2. **CH07-02**: Embeddings (Token + Positional)
3. **CH07-03**: Attention 機制深入
4. **CH07-04**: Encoder 架構與 BERT
5. **CH07-05**: Decoder 架構與自回歸生成
6. **CH07-06**: 三大架構對比與選型
7. **CH07-07**: LLM 原理 (Scaling Laws, RLHF, Prompt Engineering)
8. **CH07-08**: 實戰應用 (RAG, LangChain, Agent, 部署)

**技能樹**:
```
Transformer 基礎
    ↓
架構理解 (Encoder/Decoder/Enc-Dec)
    ↓
LLM 原理與使用
    ↓
實戰應用開發
    ↓
生產環境部署
```

### 7.4 延伸資源

**官方文檔**:
- [Hugging Face Transformers](https://huggingface.co/docs/transformers)
- [LangChain Documentation](https://python.langchain.com/)
- [OpenAI Cookbook](https://github.com/openai/openai-cookbook)

**實戰項目**:
- [LangChain Templates](https://github.com/langchain-ai/langchain/tree/master/templates)
- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT Engineer](https://github.com/gpt-engineer-org/gpt-engineer)

**社群資源**:
- [r/LocalLLaMA](https://www.reddit.com/r/LocalLLaMA/) - 本地 LLM 討論
- [Hugging Face Discord](https://discord.gg/hugging-face)
- [LangChain Discord](https://discord.gg/langchain)

---

## 🎉 恭喜完成 CH07 全部課程!

你現在已經掌握:
- ✅ Transformer 架構從零實作
- ✅ BERT/GPT/T5 三大架構原理
- ✅ LLM 的訓練與對齊
- ✅ Prompt Engineering 技巧
- ✅ RAG, LangChain, Agent 實戰
- ✅ 生產環境部署知識

**下一步建議**:
1. 實作一個完整的 RAG 問答系統
2. 嘗試微調開源 LLM (LoRA, QLoRA)
3. 探索 Multimodal LLM (Vision-Language Models)
4. 學習 LLM 安全與倫理議題

---

**課程完成時間**: `____年____月____日`  
**學習心得**: ___________________________________  
**未來計畫**: ___________________________________